In [1]:
import pandas as pd
import tqdm
import numpy as np
import codecs
import glob
import random
import matplotlib.pyplot as plt
import sys

In [3]:
liar_coefs = pd.read_csv("liar_TEST_coefs_10000feats_5gram-l1.csv", sep='\t')
kaggle_coefs = pd.read_csv("allcoefs_10000-5gram-l1_.csv", sep='\t')

In [9]:
classes = ['barely-true', 'false','half-true', 'mostly-true', 'pants-fire', 'true']
liar_coefs.columns = [s.replace('_', ' ') for s in liar_coefs.columns]
#print(coefs.isnull().values.any())
print(len(liar_coefs))
print(len(classes))
liar_coefs["class_labels"] = classes
liar_coefs.set_index("class_labels", inplace=True)
liar_coefs = liar_coefs.T
false = liar_coefs.sort_values(by='false', ascending=False)
false = false["false"]
print(false.head(10))
print(false.tail(10))

6
6
debunked        2.801328
never asked     2.619117
vaccines        2.573224
as it           2.550316
for the past    2.387442
destroyed       2.384117
regulation      2.261119
the 11          2.231280
supporter       2.072337
the crime       1.974049
Name: false, dtype: float64
was just        -1.387832
paid for        -1.389211
products        -1.441600
federal law     -1.456339
to let          -1.532681
many of         -1.550490
60 percent of   -1.660313
tax on          -1.736214
privatize       -2.073705
clinton said    -2.093048
Name: false, dtype: float64


In [10]:
# binaries liar
# find "best features" through highest coefficients of binary data


In [20]:
kaggle_coefs.columns = [s.replace('_', ' ') for s in kaggle_coefs.columns]
#coefsav = pd.DataFrame(coefs.mean(axis=1), columns=['coef'])

# true (towards label 0)
kaggle_coefs.T.sort_values(by=0, ascending=False).tail(20)

# false (towards label 1)
kaggle_coefs.T.sort_values(by=0, ascending=False).head(30)

,0
2016,0.578970
october,0.437541
november,0.379234
hillary,0.370566
anti,0.364145
share,0.312655
source,0.287841
com,0.270479
us,0.261416
year old,0.243031


In [21]:
# make common vocabulary (only shared words)


In [24]:
filename = "data/PolitiFact/FakeNewsContent/PolitiFact_Fake_1-Webpage.json"

import json
with open(filename, 'r') as f:
    datastore = json.load(f)
    
type(datastore["text"])

str

In [62]:
real = codecs.open("data/RealNewsContent.txt", 'r', 'utf-8').read().split('\n')
real = [s.lower() for s in real if len(s) > 1]
print(len(real))

fake = codecs.open("data/FakeNewsContent.txt", 'r', 'utf-8').read().split('\n')
fake = [s.lower() for s in fake if len(s) > 1]
print(len(fake))

real_labels = np.ones(len(real))
fake_labels = np.zeros(len(fake))
labels = np.concatenate((real_labels, fake_labels))
all_news = np.concatenate((real, fake))
assert len(labels) == len(all_news)


120
118


In [72]:
from sklearn.utils import shuffle
all_news, labels = shuffle(all_news,labels, random_state=42)

from sklearn.model_selection import train_test_split
FNN_train, FNN_test, FNN_train_lab, FNN_test_lab = train_test_split(all_news, labels, test_size=0.20, random_state=42)

In [74]:
len(FNN_test)

48

In [70]:
y

array([1., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 1., 1.,
       0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0., 1.,
       1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0.,
       1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 1., 0., 1., 1.,
       0., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 1.,
       1., 1., 1., 1., 1., 0., 0., 1., 0., 0., 1., 1., 1., 0., 1., 0., 1.,
       0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 0., 0., 1.,
       1., 0., 1., 1., 1., 0., 0., 1., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
       0., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1., 0.,
       1., 1., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1., 1.,
       1., 0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 1., 0.,
       0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1.,
       1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1.,
       1., 1., 1., 0., 1.